In [50]:
# with open("config.json", "r") as f_conf:   #Leo json config y lo llamo como f_conf.
#     conf=json.load(f_conf)                 # Cargo json.
#     database_filename=os.path.join('../sqlite_data',conf["sqlite_viz"]) #me devuelvo a la carpeta sqlite_data y utilizo
#                                                                         # "sqlite_viz" desde el json.

In [51]:
# conn=sqlite3.connect('../sqlite_data\\RICardo_viz.sqlite')
# c=conn.cursor()

In [52]:
# create the dataframe from a query
# df = pd.read_sql_query("SELECT * FROM flows", conn)
# df
#csv=pd.read_csv('../data/currencies.csv')
#csv=df

#csv_files = glob.glob('../data\\*.csv')
#csv_files

In [57]:
# python script to build sqlite for visualization
# coding=utf8
import subprocess
import sqlite3
import os
import json
import itertools
# import utils
import re
# import custom_exports
import shutil
import pandas as pd
import glob
import matplotlib.pyplot as plt
import numpy as np

#Set working: C:\Users\Joyas\Desktop\github\ricardo_data\database_scripts

In [53]:
#Read csv
currencies=pd.read_csv('../data/currencies.csv')
flows=pd.read_csv('../data/flows.csv')
entity_names=pd.read_csv('../data/entity_names.csv')
exchange_rates=pd.read_csv('../data/exchange_rates.csv')
expimp_spegen=pd.read_csv('../data/expimp_spegen.csv')
RICentities=pd.read_csv('../data/RICentities.csv')
RICentities_groups=pd.read_csv('../data/RICentities_groups.csv')
sources=pd.read_csv('../data/sources.csv')

C:\Users\Joyas\anaconda3\envs\ox\lib\site-packages\IPython\core\interactiveshell.py:2714: DtypeWarning: Columns (1,9,10,11,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [79]:
# exchange_rates

,year,modified_currency,rate_to_pounds,source,notes
0,1926,albanian gold franc,25.177507,StatisticalYearbookOfTheLeagueOfNations,NaN
1,1927,albanian gold franc,25.056701,StatisticalYearbookOfTheLeagueOfNations,NaN
2,1928,albanian gold franc,25.212435,StatisticalYearbookOfTheLeagueOfNations,NaN
3,1929,albanian gold franc,25.265970,StatisticalYearbookOfTheLeagueOfNations,NaN
4,1930,albanian gold franc,25.240603,StatisticalYearbookOfTheLeagueOfNations,NaN
...,...,...,...,...,...
7321,1914,brazilian gold milreis,4.659600,StatisticalAbstractForThePrincipalAndOtherFore...,1 Milreis = 2s 3d
7322,1863,argentine gold peso,4.888000,SchneiderJSchwarzerOMDenzel_WährungenDerWelt_L...,Use exchange rate in 1865.
7323,1922,canadian dollar,4.525400,HistoricalStatisticsOfCanada_SeriesJ560567,NaN
7324,1864,argentine gold peso,4.888000,SchneiderJSchwarzerOMDenzel_WährungenDerWelt_L...,Use exchange rate in 1865.


#### Cruce de tablas obtenidas de Git

In [54]:
entity_names_reporting=entity_names
entity_names_partner=entity_names

In [55]:
entity_names_reporting=entity_names_reporting.rename(columns={'original_name': "reporting"})
#entity_names_reporting
entity_names_partner=entity_names_partner.rename(columns={'original_name': "partner"})
#entity_names_partner

In [56]:
######################################################################################################
##################### Tablón que contenga reporting y partner con nombres en inglés###################
######################################################################################################

#Join1
df = pd.merge(flows, entity_names_reporting, how='left', on=["reporting"])
df=df.rename(columns={'french_name': "eliminar", 'RICname': "RICname_reporting" })

#Join2
df = pd.merge(df, entity_names_partner, how='left', on=["partner"])
df=df.rename(columns={'french_name': "eliminar2", 'RICname': "RICname_partner" })

df=df.drop(['eliminar', 'eliminar2'], axis=1)

################################################################################
##################### Tablón para convertir flujos a libras#####################
################################################################################

#Join exchangerate to currencies
df_currencies = pd.merge(currencies, exchange_rates[["rate_to_pounds","year","modified_currency"]], how='left', on=["year","modified_currency"])

#Join df_currencies to df
df= pd.merge(df, df_currencies, how='left', on=["currency","year","reporting"])
df.head(5)

,source,flow,unit,currency,year,reporting,partner,export_import,special_general,species_bullions,transport_type,statistical_period,partner_sum,world_trade_type,notes,RICname_reporting,RICname_partner,modified_currency,rate_to_pounds
0,AbstractOfReportsOfTradeOfVariousCountries,403409.0,1,livre anglaise,1857,grèce,angleterre,exp,gen,S,NaN,NaN,NaN,NaN,NaN,Greece,United Kingdom,sterling pound,1.0
1,AbstractOfReportsOfTradeOfVariousCountries,1035.0,1,livre anglaise,1857,grèce,amérique,exp,gen,S,NaN,NaN,NaN,NaN,NaN,Greece,America,sterling pound,1.0
2,AbstractOfReportsOfTradeOfVariousCountries,166264.0,1,livre anglaise,1857,grèce,autriche et autres etats allemands,exp,gen,S,NaN,NaN,NaN,NaN,NaN,Greece,Austria-Hungary & Germany,sterling pound,1.0
3,AbstractOfReportsOfTradeOfVariousCountries,10338.0,1,livre anglaise,1857,grèce,egypte,exp,gen,S,NaN,NaN,NaN,NaN,NaN,Greece,Egypt/United Arab Republic,sterling pound,1.0
4,AbstractOfReportsOfTradeOfVariousCountries,NaN,1,livre anglaise,1857,grèce,hollande et belgique*,exp,gen,S,NaN,NaN,NaN,NaN,NaN,Greece,Belgium & Netherlands,sterling pound,1.0


#### Limpieza 

In [12]:
df['year'] = df["year"].astype(str)
df['export_import'] = df["export_import"].astype(str)

# Detectar llave única de la tabla
df=df.drop(['notes'], axis=1)
df=df.drop_duplicates()
df["flow2"] = df["flow"].astype(str)
df['key1'] =  df['reporting'] + df['partner'] + df['export_import'] + df['year'] + df['flow2']+ df['source']  

df["flow"] = df["flow"].astype(str)
df['flow'] = df.flow.replace(' ','')

df=df[~df['flow'].str.contains('-')]
df=df[~df['flow'].str.contains('revoir')]
df=df[df['flow']!='nan']
df=df[df['RICname_partner']!='***NA']
df=df[df['RICname_reporting']!='***NA']
df=df[(df['unit']==1)|(df['unit']==10)|(df['unit']==1000)|(df['unit']==100000)|(df['unit']==1000000)]
df["RICname_partner"] = df["RICname_partner"].astype(str)
df=df[~df['RICname_partner'].str.contains('World')]

df["flow"] = df["flow"].astype(float)

df['flujo_unit'] = (df.flow*df.unit)/df.rate_to_pounds   #Convertir flujos a libras.

In [76]:
df

,source,flow,unit,currency,year,reporting,partner,export_import,special_general,species_bullions,transport_type,statistical_period,partner_sum,world_trade_type,RICname_reporting,RICname,flow2,key1,flujo_unit
0,AbstractOfReportsOfTradeOfVariousCountries,403409.00,1,livre anglaise,1857,grèce,angleterre,exp,gen,S,NaN,NaN,NaN,NaN,Greece,United Kingdom,403409.0,grèceangleterreexp1857403409.0AbstractOfReport...,403409.0
1,AbstractOfReportsOfTradeOfVariousCountries,1035.00,1,livre anglaise,1857,grèce,amérique,exp,gen,S,NaN,NaN,NaN,NaN,Greece,America,1035.0,grèceamériqueexp18571035.0AbstractOfReportsOfT...,1035.0
2,AbstractOfReportsOfTradeOfVariousCountries,166264.00,1,livre anglaise,1857,grèce,autriche et autres etats allemands,exp,gen,S,NaN,NaN,NaN,NaN,Greece,Austria-Hungary & Germany,166264.0,grèceautriche et autres etats allemandsexp1857...,166264.0
3,AbstractOfReportsOfTradeOfVariousCountries,10338.00,1,livre anglaise,1857,grèce,egypte,exp,gen,S,NaN,NaN,NaN,NaN,Greece,Egypt/United Arab Republic,10338.0,grèceegypteexp185710338.0AbstractOfReportsOfTr...,10338.0
5,AbstractOfReportsOfTradeOfVariousCountries,67030.00,1,livre anglaise,1857,grèce,france,exp,gen,S,NaN,NaN,NaN,NaN,Greece,France,67030.0,grècefranceexp185767030.0AbstractOfReportsOfTr...,67030.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396057,YoshikoNagano_ReexaminingTheForeignTradeStruct...,1672.14,1000,pesos,1934,philippines,oceania,imp,?,NaN,NaN,NaN,NaN,NaN,Philippines,Oceania,1672.14,philippinesoceaniaimp19341672.14YoshikoNagano_...,1672140.0
396058,YoshikoNagano_ReexaminingTheForeignTradeStruct...,3420.96,1000,pesos,1935,philippines,oceania,imp,?,NaN,NaN,NaN,NaN,NaN,Philippines,Oceania,3420.96,philippinesoceaniaimp19353420.96YoshikoNagano_...,3420960.0
396059,YoshikoNagano_ReexaminingTheForeignTradeStruct...,4045.04,1000,pesos,1936,philippines,oceania,imp,?,NaN,NaN,NaN,NaN,NaN,Philippines,Oceania,4045.04,philippinesoceaniaimp19364045.04YoshikoNagano_...,4045040.0
396060,YoshikoNagano_ReexaminingTheForeignTradeStruct...,4361.02,1000,pesos,1937,philippines,oceania,imp,?,NaN,NaN,NaN,NaN,NaN,Philippines,Oceania,4361.02,philippinesoceaniaimp19374361.02YoshikoNagano_...,4361020.0


#### Creación de tablas para generar ranking

In [48]:
#Filtro por type:países utilizando tabla RICentities 
df=df.rename(columns={'RICname_partner': "RICname"})
df3 = pd.merge(df, RICentities, how='left', on=["RICname"]) #RICentities contiene el grupo: Country, continent, etc.
df3=df3.rename(columns={'RICname': "RICname_partner"})
df3=df3[df3['type']=='country'] #Filtro solo países
df3.head(5) #DF3 contiene latitud y longitud

,source,flow,unit,currency,year,reporting,partner,export_import,special_general,species_bullions,...,key1,flujo_unit,type,continent,GPH_code,slug,part_of_country,wikidata,lat,lng
0,AbstractOfReportsOfTradeOfVariousCountries,403409.0,1,livre anglaise,1857,grèce,angleterre,exp,gen,S,...,grèceangleterreexp1857403409.0AbstractOfReport...,403409.0,country,Europe,200,UnitedKingdom,NaN,Q145,54.600000,-2.00
3,AbstractOfReportsOfTradeOfVariousCountries,10338.0,1,livre anglaise,1857,grèce,egypte,exp,gen,S,...,grèceegypteexp185710338.0AbstractOfReportsOfTr...,10338.0,country,Africa,651,EgyptUnitedArabRepublic,NaN,Q79,27.000000,29.00
4,AbstractOfReportsOfTradeOfVariousCountries,67030.0,1,livre anglaise,1857,grèce,france,exp,gen,S,...,grècefranceexp185767030.0AbstractOfReportsOfTr...,67030.0,country,Europe,220,France,NaN,Q142,47.000000,2.00
5,AbstractOfReportsOfTradeOfVariousCountries,43855.0,1,livre anglaise,1857,grèce,iles ioniennes,exp,gen,S,...,grèceiles ioniennesexp185743855.0AbstractOfRep...,43855.0,country,Europe,3349,IonianIs.,NaN,Q170295,38.500000,20.50
6,AbstractOfReportsOfTradeOfVariousCountries,5599.0,1,livre anglaise,1857,grèce,deux-siciles,exp,gen,S,...,grècedeux-sicilesexp18575599.0AbstractOfReport...,5599.0,country,Europe,329,TwoSicilies,NaN,Q180393,40.833333,14.25


In [47]:
#Calcular flujo promedio (import & export) del periodo para cada par: Reporting-Partner.
df4=df3.groupby(['RICname_reporting','RICname_partner'],as_index=False).agg({'flujo_unit':['mean']})
df4.columns=[x[0] if x[1]=='' else '_'.join(x) for x in df4.columns] #Arreglar nombre de columnas
df4=df4.sort_values(['RICname_reporting','flujo_unit_mean'],ascending=False) #Orden descendente
df4.head(5)

,RICname_reporting,RICname_partner,flujo_unit_mean
6572,Zimbabwe (Rhodesia) (Southern Rhodesia),United Kingdom,2.158675e+06
6570,Zimbabwe (Rhodesia) (Southern Rhodesia),South Africa,9.589911e+05
6573,Zimbabwe (Rhodesia) (Southern Rhodesia),United States of America,5.125029e+05
6575,Zimbabwe (Rhodesia) (Southern Rhodesia),Zambia (Northern Rhodesia),2.821289e+05
6574,Zimbabwe (Rhodesia) (Southern Rhodesia),Zaire (Kinshasa) (Belgian Congo),2.435801e+05


In [36]:
#Para quedarme con las 5 primeras tuplas con mayor flujo
df5 = df4.groupby(['RICname_reporting']).apply(lambda x: x.head(5)) 
df5 = df5.reset_index(drop=True)
df5.head(10) #DF5 contiene el flujo promedio

,RICname_reporting,RICname_partner,flujo_unit_mean
0,A Coruna,Cuba,3.818446e+04
1,A Coruna,United Kingdom,2.340927e+04
2,A Coruna,France,1.382463e+04
3,A Coruna,United States of America,1.283659e+04
4,A Coruna,Puerto Rico,1.238211e+04
5,Aden,Iraq (Mesopotamia),1.017649e+06
6,Aden,Japan,6.194570e+05
7,Aden,India,5.674104e+05
8,Aden,United Kingdom,3.202029e+05
9,Aden,Djibouti (Afar and Issas) (French Somaliland),2.670773e+05


#### Creación de tabla con coordenadas

In [34]:
# Obtengo latitud y longitud del Partner desde DF3
lat_long=df3.groupby(['RICname_partner'],as_index=False).agg({'lat':['max'],'lng':['max']})
lat_long.columns=[x[0] if x[1]=='' else '_'.join(x) for x in lat_long.columns] #Arreglar nombre de columnas
lat_long=lat_long.rename(columns={'lat_max': "lat", 'lng_max': "lng" })
lat_long.head(10)

,RICname_partner,lat,lng
0,Aden,12.800000,45.033333
1,Afghanistan,34.000000,66.000000
2,Alaska,64.000000,-150.000000
3,Albania,41.000000,20.000000
4,Aleppo,36.200000,37.150000
5,Algeria,28.000000,1.000000
6,Alhucemas,35.215000,-3.896111
7,American Samoa,-14.295833,-170.707500
8,Andaman and Nicobar Is.,11.680000,92.770000
9,Andorra,42.558333,1.555278


In [42]:
# Obtengo latitud y longitud para los Reporting desde "lat_long" a "df5"
lat_long=lat_long.rename(columns={'RICname_partner': "RICname_reporting"})
df6 = pd.merge(df5, lat_long, how='left', on=["RICname_reporting"])
df6=df6.rename(columns={'lat': "lat_reporting",'lng':'lng_reporting'})
df6.iloc[10:20,:]

,RICname_reporting,RICname_partner,flujo_unit_mean,lat_reporting,lng_reporting
10,Alaska,United States of America,3.574741e+06,64.0,-150.00
11,Alaska,Canada,2.169268e+05,64.0,-150.00
12,Albania,Italy,4.055829e+05,41.0,20.00
13,Albania,Romania,1.533028e+05,41.0,20.00
14,Albania,Greece,7.958044e+04,41.0,20.00
15,Albania,Bulgaria,6.694446e+04,41.0,20.00
16,Albania,United States of America,6.568142e+04,41.0,20.00
17,Aleppo,United Kingdom,1.030095e+05,36.2,37.15
18,Aleppo,Turkey,6.608601e+04,36.2,37.15
19,Aleppo,France,4.619206e+04,36.2,37.15


In [46]:
# Obtengo latitud y longitud para los Partner desde "lat_long" a "df6"
lat_long=lat_long.rename(columns={'RICname_reporting': "RICname_partner"})
df7= pd.merge(df6, lat_long, how='left', on=["RICname_partner"])
df7=df7.rename(columns={'lat': "lat_partner",'lng':'lng_partner'})
df7.iloc[7:20,:]

,RICname_reporting,RICname_partner,flujo_unit_mean,lat_reporting,lng_reporting,lat_partner,lng_partner
7,Aden,India,5.674104e+05,12.8,45.033333,22.000,77.000000
8,Aden,United Kingdom,3.202029e+05,12.8,45.033333,54.600,-2.000000
9,Aden,Djibouti (Afar and Issas) (French Somaliland),2.670773e+05,12.8,45.033333,11.800,42.433333
10,Alaska,United States of America,3.574741e+06,64.0,-150.000000,38.895,-77.036667
11,Alaska,Canada,2.169268e+05,64.0,-150.000000,56.000,-109.000000
12,Albania,Italy,4.055829e+05,41.0,20.000000,42.500,12.500000
13,Albania,Romania,1.533028e+05,41.0,20.000000,46.000,25.000000
14,Albania,Greece,7.958044e+04,41.0,20.000000,38.500,23.000000
15,Albania,Bulgaria,6.694446e+04,41.0,20.000000,42.750,25.500000
16,Albania,United States of America,6.568142e+04,41.0,20.000000,38.895,-77.036667


In [67]:
# paises_base=df7[~df7["dist"].isnull()]
# len(paises_base["RICname_reporting"].unique())
#df8[df8['RICname_reporting']=='United States of America']

,RICname_reporting,dist_mean,dist_sum
173,United States of America,59.46283,297.314152


In [64]:
#
# x=df7[df7["lat_reporting"].isnull()]
# x["RICname_reporting"].unique()

#### Calculo de distancia de cada país con sus 5 principales socios comerciales

In [45]:
import osmnx as ox

df7=df7[~df7["lat_reporting"].isnull()]
df7["dist"]=ox.distance.euclidean_dist_vec(df7['lat_reporting'], df7['lng_reporting'], df7['lat_partner'], df7['lng_partner'])
df8=df7.groupby(['RICname_reporting'],as_index=False).agg({'dist':['mean','sum']})
df8.columns=[x[0] if x[1]=='' else '_'.join(x) for x in df8.columns]
df8


,RICname_reporting,dist_mean,dist_sum
0,Aden,42.582385,212.911925
1,Alaska,59.467387,118.934775
2,Albania,24.291183,121.455916
3,Aleppo,27.226946,136.134729
4,Algeria,28.693478,143.467390
...,...,...,...
183,Yugoslavia/Serbia,12.032095,60.160473
184,Zaire (Kinshasa) (Belgian Congo),43.516192,217.580962
185,Zambia (Northern Rhodesia),66.709873,333.549366
186,Zanzibar,61.008757,305.043785


#### Caluculo de distancia de cada país con los 5 principales BLABLA

In [75]:
df4=df3.groupby(['RICname_reporting'],as_index=False).agg({'flow':['mean']})
df4.columns=[x[0] if x[1]=='' else '_'.join(x) for x in df4.columns]
df4=df4.sort_values(['flow_mean'],ascending=False)
df4.head(5)

,RICname_reporting,flow_mean
172,Netherlands,1.847009e+07
244,Spain,1.412114e+07
113,India,1.172931e+07
41,Bulgaria,1.144601e+07
280,United States of America,1.031244e+07


In [321]:
#len(df),df["key1"].count()
# df_1=df.drop_duplicates(['key0'], keep='last')
# repetidos=df.groupby('key2').size().loc[lambda x: x>1]
